In [1]:
import os
os.chdir('/content/drive/My Drive/Other Projects/Bollywood Lyrics/Bollywood dataset')

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
dfLyrics = pd.read_csv('lyrics.csv')

In [4]:
dfLyrics.sample()['Lyrics']

681    \n\nSaawali si raat ho\nKhamoshi ka saath ho.....
Name: Lyrics, dtype: object

In [5]:
removelist = ['\n--Male--','\n--Male1--','\n--Female--','\n--Female1--','\n--Female2--','\n--Male2--','\n--Male1 And Male2--','\x7f']
for word in removelist:
  dfLyrics.Lyrics=dfLyrics.Lyrics.str.replace(word,'')

In [6]:
dfLyrics.Lyrics

0       \n\nAaaaa Aaaaa\nAaaaaa\nAaaaaa\nMeetha Sa Ish...
1       \n\nAnkhiyon Hi Ankhiyon Mein\nRatiyaan Yeh Ka...
2       \n\nAray Peeli Chotiyaan,\nHawaeyn Sehen Si\nM...
3       \n\nPyar Itna Na Kar\nYeh Dil Jaata Hai Bhar\n...
4       Zaraa Maara Maara Sa\nJaane Kyun Dil Ye Ban Ba...
                              ...                        
1286     \n\nKaara fankaara kab aaye re\nKaara fankaar...
1287     \n\nZameen-o-aasmaan ne kya baat ki hai\nGira...
1288     \n\nMaula Wa Sallim wassalim da-iman abadan\n...
1289     \n\nBhavara\nRe ga re ga re ga re ga pa ma ga...
1290     \n\nArre udi udi udi... udi jaye..\n\nUdi udi...
Name: Lyrics, Length: 1291, dtype: object

In [7]:
dfLyrics.Lyrics.apply(lambda x: '--Fem' in x).value_counts()

False    1291
Name: Lyrics, dtype: int64

In [8]:
dfLyrics.Lyrics.apply(lambda x: '--Male' in x).value_counts()

False    1291
Name: Lyrics, dtype: int64

In [9]:
dfLyrics[dfLyrics.Lyrics.apply(lambda x: '_' in x)]
#\x7f': 83,
# 'à': 84,
# 'é': 85

,SongID,Lyrics
1226,1519,\n\nMaine suna hai tu twenty plus ho gayi\nCh...


In [10]:
dfLyrics.Lyrics = dfLyrics.Lyrics.str.replace('à','a')
dfLyrics.Lyrics = dfLyrics.Lyrics.str.replace('é','e')

In [11]:
#dfLyrics.Lyrics = dfLyrics.Lyrics.str.replace('[^\w\s]','')

In [12]:
#dfLyrics.Lyrics = dfLyrics.Lyrics.apply(lambda x:x.lower())

In [13]:
dfLyrics.Lyrics

0       \n\nAaaaa Aaaaa\nAaaaaa\nAaaaaa\nMeetha Sa Ish...
1       \n\nAnkhiyon Hi Ankhiyon Mein\nRatiyaan Yeh Ka...
2       \n\nAray Peeli Chotiyaan,\nHawaeyn Sehen Si\nM...
3       \n\nPyar Itna Na Kar\nYeh Dil Jaata Hai Bhar\n...
4       Zaraa Maara Maara Sa\nJaane Kyun Dil Ye Ban Ba...
                              ...                        
1286     \n\nKaara fankaara kab aaye re\nKaara fankaar...
1287     \n\nZameen-o-aasmaan ne kya baat ki hai\nGira...
1288     \n\nMaula Wa Sallim wassalim da-iman abadan\n...
1289     \n\nBhavara\nRe ga re ga re ga re ga pa ma ga...
1290     \n\nArre udi udi udi... udi jaye..\n\nUdi udi...
Name: Lyrics, Length: 1291, dtype: object

In [14]:
#dfLyrics.Lyrics=dfLyrics.Lyrics.str.replace(r'\d+','')

In [15]:
text = '\n\n\n\n'.join(dfLyrics.Lyrics)

In [16]:
len(text)

1698948

In [17]:
# Take a look at the first 250 characters in text
print(text[:250])



Aaaaa Aaaaa
Aaaaaa
Aaaaaa
Meetha Sa Ishq Laage
Kadvi Judayi
Yaar Mera Sacha Laage
Jhooti Khudayi
Meetha Sa Ishq Laage
Kadvi Judayi
Yaar Mera Sacha Laage
Jhooti Khudayi
Chandni Ne Tan Pe Mere
Chaadar Bhichayee
Odha Jo Tune Mujhko
Saans Laut Aayi
Cha


In [18]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

86 unique characters


In [19]:
char2idx = {u:i for i,u in enumerate(vocab)}

In [20]:
idx2char = np.array(vocab)

In [21]:
text_as_int = np.array([char2idx[x] for x in text])

In [22]:
len(text_as_int)

1698948

In [23]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '&' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '*' :   9,
  ',' :  10,
  '-' :  11,
  '.' :  12,
  '/' :  13,
  '0' :  14,
  '1' :  15,
  '2' :  16,
  '3' :  17,
  '4' :  18,
  '5' :  19,
  ...
}


In [24]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\n\nAaaaa Aaaaa' ---- characters mapped to int ---- > [ 1  1 27 56 56 56 56  2 27 56 56 56 56]


In [25]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [26]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [27]:
char_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [28]:
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])





A
a
a


In [29]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [30]:
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\n\nAaaaa Aaaaa\nAaaaaa\nAaaaaa\nMeetha Sa Ishq Laage\nKadvi Judayi\nYaar Mera Sacha Laage\nJhooti Khudayi\nMe'
'etha Sa Ishq Laage\nKadvi Judayi\nYaar Mera Sacha Laage\nJhooti Khudayi\nChandni Ne Tan Pe Mere\nChaadar B'
'hichayee\nOdha Jo Tune Mujhko\nSaans Laut Aayi\nChandni Ne Tan Pe Mere\nChaadar Bhichayee\nOdha Jo Tune Mu'
'jhko\nSaans Laut Aayi\nOhooooo Ohhoooo\nYeh Aalam Hai Ishq Ishq Aalam Hai\nOhooooo Ohhoooo\nYeh Aalam Hai '
'Ishq Ishq Aalam Hai\n\nMehndi Ka Rang Jhat Se Gehra Gaya\nEk Pal Mein Haaye Kya Hua\nPayalo Ka Shor Jaane'


In [31]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [32]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\n\nAaaaa Aaaaa\nAaaaaa\nAaaaaa\nMeetha Sa Ishq Laage\nKadvi Judayi\nYaar Mera Sacha Laage\nJhooti Khudayi\nM'
Target data: '\nAaaaa Aaaaa\nAaaaaa\nAaaaaa\nMeetha Sa Ishq Laage\nKadvi Judayi\nYaar Mera Sacha Laage\nJhooti Khudayi\nMe'


In [33]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 1 ('\n')
  expected output: 1 ('\n')
Step    1
  input: 1 ('\n')
  expected output: 27 ('A')
Step    2
  input: 27 ('A')
  expected output: 56 ('a')
Step    3
  input: 56 ('a')
  expected output: 56 ('a')
Step    4
  input: 56 ('a')
  expected output: 56 ('a')


In [34]:
# Batch size
BATCH_SIZE = 64

In [35]:
# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

In [36]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [37]:
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [38]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [39]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [40]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [41]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 86) # (batch_size, sequence_length, vocab_size)


In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           22016     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 86)            88150     
Total params: 4,048,470
Trainable params: 4,048,470
Non-trainable params: 0
_________________________________________________________________


In [43]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [44]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'shaaron pe nachega saara jahan\n\nTak dhina dhin.. Hey aladin\nBadli duniya ke aaya naya din\nTak dhina '

Next Char Predictions: 
 'dWw3T,-9"9v=oyoeK|.9T1/EJ‘…bLiI7d,7hkG.X2a3ztje5kR‘Rb*KakO‘LT\'Xu-Mt,M0d|XS\'H‘\t….L9jH5AU9\twWbdEXKxt]\''


In [45]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 86)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.453406


In [46]:
model.compile(optimizer='adam', loss=loss)

In [47]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [48]:
EPOCHS=20

In [49]:
#history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [50]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [51]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [52]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            22016     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 86)             88150     
Total params: 4,048,470
Trainable params: 4,048,470
Non-trainable params: 0
_________________________________________________________________


In [53]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [55]:
print(generate_text(model, start_string=u'jab se tere naina'))

jab se tere naina meethe ghaat ka
Mera dil hai ptak raha

Are dekho bemarwaar kyun kya jaane
Dil ko mila pal pal barse tujhe, main hoon bewajaa..
Tujhko plene mein chupi
Dekho muftie endri si namkeen, jine le jaaye, tujhko main dekhun
Baahon mei leke jaanay baithe baithe
Chaand ki chhori yeh agan-ve

Ga raa ringa ding dang aaieye...





 

Kadam kya basti, jesi hai pariye
Udaash teri aadat hai
Tu dikhmani, dont it up!
(i wanna hila woh,
Ude utha ho ya koi jadugari

Teri aankhon ka sameen hai
Tere bin kahan pe hai, tum zindag zameen pe-chalti hai
Pehle ka rang jo hai mujhse
Rubahu tu hmm

Aehn meri
Tu kheech meri photo. le tu


Tere bin mein wahan jaane kya
Fikar dhadke jiyein aaye, vaare dil ko
Ho gaya s, leke chal bhave
O to chaska ghumaar
Khaali huaa aashiq
Tum tip top friday)
Ha ha laga kehna 
Pyaar waalay chadhayegi badley gaadi doubt hai
Uski khushi ho saam de
Raah mein dekhun na baithe baithe
Chahe toh jaadu karte hain to
Darge, hulce hain adhoore sexy
Hey.'itbon'mein hai jaisa,